# Configure visualization with AnnData-Zarr

## Motivation

We can configure an interactive visualization that loads the data from the AnnData-Zarr store (the result of the `01_convert_data.ipynb` notebook) using Vitessce.

In [2]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
)
from os.path import join

## Instantiate a `VitessceConfig` object

We begin the configuration process by creating an object using the `VitessceConfig` constructor. This takes three parameters:
- `schema_version`: the schema version that the configuration should conform to. Valid values can be found at http://vitessce.io/docs/view-config-json/#version. The current most recent version `1.0.15` (as of 1/16/2023) is used in this tutorial.
- `name`: a name for the configuration.
- `description` (optional): a brief description of the configuration. This will appear in the `cm.DESCRIPTION` view (if it is added to the configuration).

__Note__: in these tutorials and in the Vitessce Python package documentation, we typically store the config instance in a variable called `vc`, but it can really be named anything.

In [3]:
vc = VitessceConfig(schema_version="1.0.15", name='Habib et al', description='COVID-19 Healthy Donor Brain')

## Add data

To add data to the configuration, we first run [add_dataset](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.add_dataset) which takes the dataset `name` as a parameter.

This returns a new `VitessceConfigDataset` instance. We can add objects which represent local data such as AnnData stores by running [add_object](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfigDataset.add_object) on the dataset instance. To enable multiple `add_object` calls to be chained together, the `add_object` function also returns the `VitessceConfigDataset` instance.

We will store the `VitessceConfigDataset` instance in a variable (`dataset`) to use later when configuring views.

__Note__: the functions used in the following cells `.add_dataset`, `.add_object`, and `.add_view` have side effects (i.e., they modify the `vc` object). Running these cells more than once on the same `vc` instance may result in an unexpected configuration, so be sure to run the cells in order starting from the initial `vc = VitessceConfig(...)` cell.

In [4]:
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        adata_path=join("processed_data", "habib17.zarr"),
        obs_embedding_paths=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_paths=[["obs/CoarseCellType", "obs/CellType"]],
        obs_set_names=["Cell Type"],
        obs_feature_matrix_path="X"
    )
)

## Add views

Next, we configure the visualization and controller views of interest. Based on the data available in the dataset we added to the configuration above (i.e., the `habib17` AnnData object), we might want to add the following view types:
- a `SCATTERPLOT` view, to visualize the UMAP dimensionality reduction results
- a cell set manager, using the `OBS_SETS` view type (remember that our observations represent cells)
- a list for selecting genes, using the `FEATURE_LIST` view type (remember that our features represent genes)
- a `HEATMAP` view, to visualize the gene expression matrix
- a  cell set sizes bar chart, using the `OBS_SET_SIZES` view type

In [5]:
scatterplot = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
genes = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(vt.HEATMAP, dataset=dataset)
cell_set_sizes = vc.add_view(vt.OBS_SET_SIZES, dataset=dataset)

## Arrange views

We use the [layout](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.layout) function to specify how the views that we configured above are organized in a grid arrangement. The pipe character `|` and backslash character `/` are "magic" syntax for [horizontal](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.hconcat) and [vertical](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.vconcat) concatenation, respectively. When using the `layout` function, every view that was added to the config (i.e., the objects returned by the `add_view` calls above) should appear somewhere in the concatenation of views passed as the parameter.

__Note__: for more control over the layout, the `x`, `y`, `w` (width), and `h` (height) parameters (not used in this notebook) of the `add_view` function can be used instead of `layout`. Note that the grid has 12 columns and 12 rows, and the x/y/w/h values must be integers.

In [6]:
vc.layout((scatterplot | cell_sets) / (heatmap | (genes | cell_set_sizes)));

## Render the widget into the notebook

To render the interactive visualization into the notebook, we run the [widget](https://vitessce.github.io/vitessce-python/api_config.html#vitessce.config.VitessceConfig.widget) function. 

### What is happening behind the scenes
Because we added local data to the Vitessce configuration (by running `add_object` with an `AnnDataWrapper` object pointing to a local AnnData-Zarr store in `adata_path`), rendering the widget will also result in the local data being served over HTTP on `localhost`. The exact port for the development server can be configured using the `port` parameter of the `widget` function.

In [21]:
vw = vc.widget()
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Habib et al', 'description': 'COVID-19 Healthy Donor Brai…

To check out the interactive visualization in a larger viewport, you may want to use the `web_app` function to open the visualization in a new browser tab.

__Note__: This does not work in Google Chrome ([more details](https://github.com/vitessce/vitessce/issues/1089)) but should work in Firefox and Safari. Also note that this URL is temporary and will only work while the local data server is running (the server will stop when the Python process ends for the notebook, or when `data_server.stop_all` is run - see Cleanup section below).

In [8]:
vc.web_app()

'http://vitessce.io/#?theme=light&url=data:,%7B%22version%22%3A+%221.0.15%22%2C+%22name%22%3A+%22Habib+et+al%22%2C+%22description%22%3A+%22COVID-19+Healthy+Donor+Brain%22%2C+%22datasets%22%3A+%5B%7B%22uid%22%3A+%22A%22%2C+%22name%22%3A+%22Brain%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22anndata.zarr%22%2C+%22url%22%3A+%22http%3A%2F%2Flocalhost%3A8001%2FA%2F0%2F8ee00bf5-4f00-4d8d-b877-e01f5273399a%22%2C+%22options%22%3A+%7B%22obsEmbedding%22%3A+%5B%7B%22path%22%3A+%22obsm%2FX_umap%22%2C+%22dims%22%3A+%5B0%2C+1%5D%2C+%22embeddingType%22%3A+%22UMAP%22%7D%5D%2C+%22obsSets%22%3A+%5B%7B%22name%22%3A+%22Cell+Type%22%2C+%22path%22%3A+%5B%22obs%2FCoarseCellType%22%2C+%22obs%2FCellType%22%5D%7D%5D%2C+%22obsFeatureMatrix%22%3A+%7B%22path%22%3A+%22X%22%7D%7D%7D%5D%7D%5D%2C+%22coordinationSpace%22%3A+%7B%22dataset%22%3A+%7B%22A%22%3A+%22A%22%7D%2C+%22embeddingType%22%3A+%7B%22A%22%3A+%22UMAP%22%7D%7D%2C+%22layout%22%3A+%5B%7B%22component%22%3A+%22scatterplot%22%2C+%22coordinationScopes%22%3A+%

## Cleanup

While not necessary, you may want to ensure that any local servers are stopped, or you may want to remove the widget from the notebook.

In [26]:
# Remove the widget
vw.close()
# or, remove all widgets
vw.close_all()

In [23]:
from vitessce import data_server

In [24]:
# Stop all running local data servers
data_server.stop_all()